In [15]:
import os
import warnings
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

SEED = 42 # Muito importante manter a SEED igual em todos os modelos para garantir a consistência dos dados no ensemble
FOLDS = 5 # Muito importante manter o mesmo número de FOLDS em todos os modelos para garantir a consistência dos dados no ensemble

In [2]:
sintetico = pd.read_csv('../../src/train/train.csv', index_col='id')
original = pd.read_csv('../../src/train/original.csv')
test = pd.read_csv('../../src/test/test.csv', index_col='id')

train = pd.concat([sintetico, original], ignore_index=True)

initial_features = list(test.columns)

In [3]:
cat_features = ['Marital status', 'Application mode', 'Course',
                'Previous qualification', 'Nacionality', "Mother's qualification", 
                "Father's qualification", "Mother's occupation",
                "Father's occupation"]

for feature in cat_features:
    for df in [train, test]:
        df[feature] = df[feature].astype('category')

num_features = df.select_dtypes(include=['int64', 'float64']).columns

In [4]:
def cross_validation(model, X, y, encoder, scoring=accuracy_score, k=FOLDS):

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=SEED)

    scores = []
    out_of_fold = []
    
    for i, (train_index, val_index) in enumerate(skf.split(X, y)):
        print(f"Fold {i + 1}")
        
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]

        X_val = X.iloc[val_index]
        y_val = y.iloc[val_index]
        
        model.fit(X_train, y_train)
        probabilidades = model.predict_proba(X_val)

        # Recuperar a predição final a partir das probabilidades
        indices_predicoes = np.argmax(probabilidades, axis=1)
        classes_preditas = model.classes_[indices_predicoes]

        score = scoring(y_val, classes_preditas)

        scores.append(score)

        true_label = pd.Series(y_val.values, name='true')

        pred_label_df = pd.DataFrame(probabilidades)

        oof_pred = pd.concat([pred_label_df, true_label], axis=1, ignore_index=True)
        oof_pred.columns = [f'pred_{encoder[model.classes_[0]]}', f'pred_{encoder[model.classes_[1]]}', f'pred_{encoder[model.classes_[2]]}', 'true']

        out_of_fold.append(oof_pred)

    print(f"Score: {np.mean(scores)}")
    
    return scores, out_of_fold

In [5]:
def save_oof(oof):

    os.makedirs('oof', exist_ok=True)

    for i, fold in enumerate(oof):
        fold.to_csv(f'oof/fold_{i+1}.csv', index=False)

In [6]:
y = train['Target'].replace({"Enrolled": 0, "Graduate": 1, "Dropout": 2})
X = train[initial_features]

In [7]:
encoder = {0: "Enrolled", 1: "Graduate", 2: "Dropout"}

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
    ])

cat = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier()),
])

In [9]:
scores,oof = cross_validation(cat, X, y, encoder)

Fold 1
Learning rate set to 0.097881
0:	learn: 0.9957865	total: 61ms	remaining: 1m
1:	learn: 0.9172033	total: 74.2ms	remaining: 37s
2:	learn: 0.8565766	total: 89.1ms	remaining: 29.6s
3:	learn: 0.8053204	total: 101ms	remaining: 25.2s
4:	learn: 0.7618755	total: 114ms	remaining: 22.6s
5:	learn: 0.7257481	total: 127ms	remaining: 21s
6:	learn: 0.6951589	total: 139ms	remaining: 19.8s
7:	learn: 0.6687095	total: 153ms	remaining: 18.9s
8:	learn: 0.6467005	total: 165ms	remaining: 18.2s
9:	learn: 0.6275694	total: 178ms	remaining: 17.6s
10:	learn: 0.6105136	total: 193ms	remaining: 17.3s
11:	learn: 0.5958180	total: 206ms	remaining: 17s
12:	learn: 0.5827742	total: 219ms	remaining: 16.6s
13:	learn: 0.5707986	total: 233ms	remaining: 16.4s
14:	learn: 0.5604127	total: 246ms	remaining: 16.1s
15:	learn: 0.5514776	total: 259ms	remaining: 15.9s
16:	learn: 0.5439288	total: 272ms	remaining: 15.7s
17:	learn: 0.5364292	total: 285ms	remaining: 15.5s
18:	learn: 0.5299926	total: 298ms	remaining: 15.4s
19:	learn: 0

In [10]:
for fold in oof:
    fold['true'] = fold['true'].replace(encoder)

save_oof(oof)

In [27]:
# Função objetivo para a otimização
def objective(trial: Trial, X, y):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 2000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
        'depth': trial.suggest_int('depth', 4, 12),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 100),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'boosting_type': trial.suggest_categorical('boosting_type', ['Plain']),
        'task_type': 'CPU',
        'loss_function': 'MultiClass'
    }

    # O CatBoost não suporta bootstrap_type='Bayesian' com boosting_type='Plain'
    if params['bootstrap_type'] == 'Bayesian' and params['boosting_type'] == 'Plain':
        params['bootstrap_type'] = 'Bernoulli'

    cat = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier(**params, verbose=0)),
                    ])
    
    # # Split dos dados
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

    # cat.fit(X_train, y_train, classifier__verbose=0)

    # # Predição e avaliação
    # preds = cat.predict(X_valid)
    # accuracy = accuracy_score(y_valid, preds)
    # return accuracy

    scores,_ = cross_validation(cat, X, y, encoder, k=3)
    accuracy = np.mean(scores)


    return accuracy

In [28]:
warnings.filterwarnings("ignore")

# Configuração do estudo
study = optuna.create_study(direction='maximize', sampler=TPESampler(), pruner=MedianPruner())
study.optimize(lambda trial: objective(trial, X, y), n_trials=1000)

[I 2024-06-12 13:50:36,871] A new study created in memory with name: no-name-143cc7e9-83ef-4e4c-9c98-f0386c9cd088


Fold 1
Fold 2
Fold 3


[I 2024-06-12 13:56:15,839] Trial 0 finished with value: 0.7870326771879217 and parameters: {'iterations': 1916, 'learning_rate': 3.17336115255792e-05, 'depth': 10, 'l2_leaf_reg': 0.014461547077601224, 'bootstrap_type': 'Bayesian', 'od_type': 'Iter', 'random_strength': 12, 'border_count': 129, 'boosting_type': 'Plain'}. Best is trial 0 with value: 0.7870326771879217.


Score: 0.7870326771879217
Fold 1
Fold 2
Fold 3


[I 2024-06-12 13:57:11,852] Trial 1 finished with value: 0.8286427406332585 and parameters: {'iterations': 415, 'learning_rate': 0.061555032009411044, 'depth': 9, 'l2_leaf_reg': 0.00269355763891032, 'bootstrap_type': 'MVS', 'od_type': 'IncToDec', 'random_strength': 96, 'border_count': 60, 'boosting_type': 'Plain'}. Best is trial 1 with value: 0.8286427406332585.


...

Score: 0.7805959843782482
Fold 1
Fold 2
Fold 3


[I 2024-06-12 22:33:57,581] Trial 501 finished with value: 0.8296434691016632 and parameters: {'iterations': 1764, 'learning_rate': 0.021290814926589584, 'depth': 6, 'l2_leaf_reg': 0.0003201251914838142, 'bootstrap_type': 'Bayesian', 'od_type': 'Iter', 'random_strength': 45, 'border_count': 99, 'boosting_type': 'Plain'}. Best is trial 309 with value: 0.8325344175289153.


Score: 0.8296434691016632
Fold 1
Fold 2
Fold 3


[W 2024-06-12 22:34:28,893] Trial 502 failed with parameters: {'iterations': 1714, 'learning_rate': 0.04204947838729431, 'depth': 5, 'l2_leaf_reg': 0.002214293491983608, 'bootstrap_type': 'Bernoulli', 'od_type': 'IncToDec', 'random_strength': 40, 'border_count': 68, 'boosting_type': 'Plain'} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_506260/2713703381.py", line 5, in <lambda>
    study.optimize(lambda trial: objective(trial, X, y), n_trials=1000)
  File "/tmp/ipykernel_506260/3577745131.py", line 36, in objective
    scores,_ = cross_validation(cat, X, y, encoder, k=3)
  File "/tmp/ipykernel_506260/1156578783.py", line 17, in cross_validation
    model.fit(X_train, y_train)
  File "/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/pipel

KeyboardInterrupt: 

In [29]:
# Melhor conjunto de hiperparâmetros
print(f"Best trial: {study.best_trial.number}")
print(f"Best value: {study.best_value}")
print(f"Best params: {study.best_trial.params}")

Best trial: 309
Best value: 0.8325344175289153
Best params: {'iterations': 1740, 'learning_rate': 0.04505281135910402, 'depth': 6, 'l2_leaf_reg': 0.07065084253966697, 'bootstrap_type': 'Bernoulli', 'od_type': 'IncToDec', 'random_strength': 40, 'border_count': 95, 'boosting_type': 'Plain'}


In [42]:
cat_best_params = {'iterations': 1740, 'learning_rate': 0.04505281135910402, 'depth': 6, 'l2_leaf_reg': 0.07065084253966697, 'bootstrap_type': 'Bernoulli', 'od_type': 'IncToDec', 'random_strength': 40, 'border_count': 95, 'boosting_type': 'Plain'}
# cat_best_params = {'iterations': 956, 'learning_rate': 0.0450347941806318, 'depth': 7, 'l2_leaf_reg': 2.419872461609025e-05, 'bootstrap_type': 'MVS', 'od_type': 'IncToDec', 'random_strength': 6, 'border_count': 132, 'boosting_type': 'Plain'}

In [43]:
cat = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier(**cat_best_params, verbose=0)),
])

In [44]:
scores,oof = cross_validation(cat, X, y, encoder)

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Score: 0.8321761308455409


In [41]:
scores

[0.8307492741985298,
 0.8321699919698561,
 0.8302446256486287,
 0.82981220657277,
 0.8335186557944156]

In [45]:
scores

[0.8322935326456236,
 0.8316758292667861,
 0.8320360761057574,
 0.830491722263405,
 0.834383493946133]

In [46]:
for fold in oof:
    fold['true'] = fold['true'].replace(encoder)

save_oof(oof)